# PlotChiPT by Louis

In [4]:
!cd /content/google_drive/MyDrive
!unzip -q data.zip

### Import and mount drive

In [1]:
#import libraries
import numpy , os , glob , cv2
from tensorflow import *
from google.colab import drive

In [2]:
try:
  drive.mount("/content/google_drive")
  os.chdir("google_drive/MyDrive")
except BaseException:
  print("Cannot find drive")

Mounted at /content/google_drive


## Create dataset and training







In [ ]:
#path for dataset
path ="/content/google_drive/MyDrive/dataset_xy/"

try:
  os.chdir(path)
except BaseException:
  print("chdir error")
def get_current_xy(file_name , len , startvar):
    tem_list = []
    var = ""
    for i in file_name:
        tem_list.append(i)
    for i in range(len):
        var += tem_list[i+startvar]
    return var

#read img
file_list = glob.glob("*.jpg")
img_array = numpy.zeros((len(file_list),224,224,3))
xy_array = numpy.zeros((len(file_list) , 2))
#print(numpy.array2string(xy_array, precision=3, separator=',',suppress_small=True))
times = 0
for i in file_list:
  img = cv2.imread(i)
  xy_array[times , 0] , xy_array[times , 1] = (int(get_current_xy(i , 3 , 3)))/224 , (int(get_current_xy(i , 3 , 7))/224)
  img_array[times] = (img)
  times += 1
#print(numpy.array2string(xy_array, precision=3, separator=',',suppress_small=True))
#print(numpy.array2string(img_array, precision=3, separator=',',suppress_small=True))

In [ ]:
from tensorflow.python.ops.math_ops import Sigmoid
from keras.models.cloning import Sequential
#This part is copy from TensorFlow official website
from keras.layers import *
from keras.models import Model


model = Sequential()
model.add(InputLayer(input_shape=(224,224,3)))
model.add(Conv2D(2 , 6 , activation="relu"))
model.add(Conv2D(2 , 4 , activation="relu"))
model.add(Conv2D(2 , 2 , activation="relu"))
model.add(Conv2D(1 , 1 , activation="relu"))
model.add(Flatten())
model.add(Dense(units=256 , activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(units=64 , activation="relu"))
model.add(Dense(units=2 , activation="sigmoid"))
model.summary()
model.compile(
    optimizer="adam",
    loss='mean_squared_error',
    metrics=['accuracy'])

model.fit(img_array , xy_array , epochs = 50)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 219, 219, 2)       218       
                                                                 
 conv2d_1 (Conv2D)           (None, 216, 216, 2)       66        
                                                                 
 conv2d_2 (Conv2D)           (None, 215, 215, 2)       18        
                                                                 
 conv2d_3 (Conv2D)           (None, 215, 215, 1)       3         
                                                                 
 flatten (Flatten)           (None, 46225)             0         
                                                                 
 dense (Dense)               (None, 256)               11833856  
                                                                 
 dropout (Dropout)           (None, 256)               0

In [ ]:
os.chdir("/content/google_drive/MyDrive")

model.save("PlotChiPT2 - 0.2 .h5")

## Use model to plot

In [ ]:
try :
  os.chdir("/content/google_drive/MyDrive")
except BaseException:
  print("chdir error")

In [ ]:
dataset_path = "/content/google_drive/MyDrive/JetBot_Dataset_DL/report_by_ak_16dec/dataset_akreplottrash"
model_path = "/content/google_drive/MyDrive"
model_name = "/content/google_drive/MyDrive/PlotChiPT2 - 0.2 .h5"

def str2list(input_str):
  tem_list = []
  for i in input_str:
    tem_list.append(i)
  return tem_list

def shift(target_len , var):
  tem_list = []
  for i in range(target_len - len(var)):
    tem_list.append("0")
  for i in var:
    tem_list.append(i)
  return tem_list

def rename(img_name,x,y,xy_len,start_x,start_y):
  tem_list = []
  tem_str = ""
  tem_list = str2list(img_name)
  x , y = shift(3 , str(x)) , shift(3 , str(y))
  for i in range(xy_len):
    tem_list[(start_x)+i] = x[i]
  for i in range(xy_len):
    tem_list[(start_y)+i] = y[i]
  tem_str = "".join(tem_list)
  print(tem_str)
  os.rename(img_name , tem_str)

from keras.models import load_model

#try:
#  os.chdir() #Model load
#except BaseException:
#  print("Can't cd directory")
model = load_model(filepath=model_name)
try:
  os.chdir(dataset_path) #Dataset load
except BaseException:
  print("Can't cd directory")
file_list = glob.glob("*.jpg")
for i in file_list:
  img_pre = numpy.zeros((1,224,224,3))
  img = cv2.imread(i)
  img_pre[0]=img
  plot = model.predict(img_pre)*224
  x , y = int(plot[0,0]) , int(plot[0,1])
  rename(i , x , y , 3 , 3 , 7)
  print(i,"old\n")
  #print(numpy.array2string(plot, precision=3, separator=',',suppress_small=True))
  #print(str(x) , str(y),"\n")

In [ ]:
!cd /content/google_drive/MyDrive/
!unzip /content/google_drive/MyDrive/data.zip